In [1]:
import pywavefront

# データ読み込み

In [2]:
data = 'data/sample_data.obj'

In [3]:
scene = pywavefront.Wavefront(data)

In [4]:
len(scene.vertices)

45798

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(scene.vertices,columns=['x','y','z','r','g','b'])

In [7]:
df.describe()

,x,y,z,r,g,b
count,45798.000000,45798.000000,45798.000000,45798.000000,45798.000000,45798.000000
mean,-0.012032,0.056997,-0.000050,0.374957,0.320720,0.254689
std,0.150060,0.472951,0.080875,0.187908,0.155195,0.102629
min,-0.344800,-0.929400,-0.187100,0.000000,0.000000,0.000000
25%,-0.140600,-0.278775,-0.070300,0.184300,0.170025,0.163800
50%,-0.003700,0.070300,-0.000200,0.417500,0.334600,0.263400
75%,0.109400,0.414100,0.067200,0.536000,0.448700,0.334700
max,0.284200,0.947500,0.153900,0.755300,0.674000,0.533100


# `(x,y,z)`座標を0起点に変換

In [9]:
idxs = ['x','y','z']
df[idxs] = df[idxs] - df[idxs].apply(min)
'''
for idx in idxs:
    df[idx] = df[idx] - min(df[idx])
'''
df[idxs].describe()

,x,y,z
count,45798.000000,45798.000000,45798.000000
mean,0.332768,0.986397,0.187050
std,0.150060,0.472951,0.080875
min,0.000000,0.000000,0.000000
25%,0.204200,0.650625,0.116800
50%,0.341100,0.999700,0.186900
75%,0.454200,1.343500,0.254300
max,0.629000,1.876900,0.341000


# `(x,y,z)`座標の拡大倍率を決定する

In [21]:
%%time
max_series = df[idxs].apply(max)
max_val = max_series.max()
max_idx = max_series.idxmax()
'''
for idx in idxs:
    if max_val < max(df[idx]):
        max_val = max(df[idx])
        max_idx = idx
'''
print('col:',max_idx)
print('max_val:',max_val)

col: y
max_val: 1.8769
CPU times: user 21.4 ms, sys: 1.96 ms, total: 23.4 ms
Wall time: 25.5 ms


In [30]:
HEIGHT_MAX = 256
WIDTH_MAX = 256
if max_idx in ['x','z']:
    zoom_ratio = WIDTH_MAX / max_val
else:
    zoom_ratio = HEIGHT_MAX / max_val
print('zoom_ratio:',zoom_ratio)

zoom_ratio: 1.003921568627451


In [14]:
for idx in idxs:
    df[idx] = df[idx] * zoom_ratio
df[idxs].describe()

,x,y,z
count,45798.000000,45798.000000,45798.000000
mean,45.387956,134.539772,25.512647
std,20.467400,64.508159,11.030891
min,0.000000,0.000000,0.000000
25%,27.851883,88.742075,15.930950
50%,46.524375,136.354201,25.492248
75%,61.950663,183.246843,34.685279
max,85.792530,256.000000,46.510736


# 外接矩形領域の始点終点

In [15]:
bounds_s = (0,0,0)
bounds_e = (max(df['x']),max(df['y']),max(df['z']))
print(bounds_s,bounds_e)

(0, 0, 0) (85.79253023602749, 255.99999999999997, 46.510735787735086)


# 色情報を復元
値が`1.0`の場合に最大値(`255`)となるように変換  
※これであってるのかは不明

In [16]:
COLOR_MAX = 255
colors = ['r','g','b']
for color in colors:
    df[color] = df[color] * COLOR_MAX
df[colors].describe()

,r,g,b
count,45798.000000,45798.000000,45798.000000
mean,95.614076,81.783594,64.945790
std,47.916463,39.574685,26.170374
min,0.000000,0.000000,0.000000
25%,46.996500,43.356375,41.769000
50%,106.462500,85.323000,67.167000
75%,136.680000,114.418500,85.348500
max,192.601500,171.870000,135.940500


In [26]:
df.describe()

,x,y,z,r,g,b
count,45798.000000,45798.000000,45798.000000,45798.000000,45798.000000,45798.000000
mean,44.887178,134.040264,24.999061,95.614076,81.783594,64.945790
std,20.483990,64.503953,11.047317,47.916463,39.574685,26.170374
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.000000,88.000000,15.000000,46.996500,43.356375,41.769000
50%,46.000000,136.000000,25.000000,106.462500,85.323000,67.167000
75%,61.000000,183.000000,34.000000,136.680000,114.418500,85.348500
max,85.000000,255.000000,46.000000,192.601500,171.870000,135.940500


# 各ボクセルの色の代表値を決める

In [17]:
import numpy as np
from math import ceil
typycal_colors = np.zeros((ceil(bounds_e[0]),ceil(bounds_e[1]),ceil(bounds_e[2]),3))
typycal_colors.shape

(86, 256, 47, 3)

# 以下は計算の高速化をすること

In [18]:
from math import floor
for idx in idxs:
    df[idx] = np.floor(df[idx])

In [19]:
from tqdm import tqdm
for x in tqdm(range(ceil(bounds_e[0]))):
    #for y in tqdm(range(ceil(bounds_e[1]))):
    for y in range(ceil(bounds_e[1])):
        #for z in tqdm(range(ceil(bounds_e[2]))):
        for z in range(ceil(bounds_e[2])):
            target = df[(df['x'] == x) & (df['y'] == y) & (df['z'] == z)]
            if len(target) == 0:
                continue
            typycal_colors[x,y,z] = [target['r'].mean(),target['g'].mean(),target['b'].mean()]
typycal_colors

100%|██████████| 86/86 [17:09<00:00, 11.97s/it]


array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 

In [20]:
'''
for x in range(ceil(bounds_e[0])):
    for y in range(ceil(bounds_e[1])):
        for z in range(ceil(bounds_e[2])):
            target = df[(df['x'] >= x) & (df['x'] < x+1) & (df['y'] >= y) & (df['y'] < y+1) & (df['z'] >= z) & (df['z'] < z+1)]
            typycal_colors[x,y,z] = [target['r'].mean(),target['g'].mean(),target['b'].mean()]
typycal_colors
'''

"\nfor x in range(ceil(bounds_e[0])):\n    for y in range(ceil(bounds_e[1])):\n        for z in range(ceil(bounds_e[2])):\n            target = df[(df['x'] >= x) & (df['x'] < x+1) & (df['y'] >= y) & (df['y'] < y+1) & (df['z'] >= z) & (df['z'] < z+1)]\n            typycal_colors[x,y,z] = [target['r'].mean(),target['g'].mean(),target['b'].mean()]\ntypycal_colors\n"

In [21]:
target = df[(df['x'] == 0) & (df['y'] == 109) & (df['z'] == 29)]
target

,x,y,z,r,g,b
0,0.0,109.0,29.0,116.3565,85.6035,66.9120
1,0.0,109.0,29.0,116.0760,85.3740,66.7335
2,0.0,109.0,29.0,115.9740,85.2465,66.6315


In [22]:
[target['r'].mean(),target['g'].mean(),target['b'].mean()]

[116.1355, 85.408, 66.759]

In [23]:
#typycal_colors[0,0,0] = [target['r'].mean(),target['g'].mean(),target['b'].mean()]
#typycal_colors[0,0,0]

In [25]:
sum(typycal_colors)

array([[[  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        ...,
        [  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ]],

       [[  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        ...,
        [  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ]],

       [[  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        ...,
        [536.60925, 431.44725, 380.70225],
        [  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ]],

       ...,

       [[  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        [  0.     ,   0.     ,   0.     ],
        ...,
        [  0.     ,   0.  

In [31]:
df[idxs]

,x,y,z
0,0.0,109.0,29.0
1,0.0,109.0,29.0
2,0.0,109.0,29.0
3,0.0,109.0,30.0
4,0.0,110.0,28.0
...,...,...,...
45793,85.0,176.0,20.0
45794,85.0,176.0,21.0
45795,85.0,177.0,18.0
45796,85.0,177.0,19.0
